In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from torch.utils.data import DataLoader
from torch import nn
import torch
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

from data_prep import fetch_data, interpolate, set_target, backstep_columns, train_test_split
from model_utils import BasicData, train_net, pred_accuracy, LSTM

In [6]:
# Prep Data
df = fetch_data(num_currencies=2)
df = interpolate(df)
df = set_target(df, 1)
targets = df['target']
features = df.drop('target', axis=1)


# scaling
scaler = MinMaxScaler()
scaled_features = pd.DataFrame(scaler.fit_transform(features), columns = features.columns)

# shaping
backsteps = 10
shaped_features = backstep_columns(scaled_features, ['EURO', 'POUND'], backsteps)
targets = targets[backsteps:]
assert shaped_features.shape[0] == targets.shape[0]

In [ ]:
model = LSTM(2, 5, 5, 1)
accuracies = []
for i in range(120, len(shaped_features)-30, 30):
    X_train, X_test, y_train, y_test = train_test_split(
        shaped_features, targets,
        test_window=[i, i+30]
    )
    training_data = BasicData(X_train, y_train)
    test_data = BasicData(X_test, y_test)
    train_loader = DataLoader(
        training_data, batch_size=64, shuffle=True, drop_last=True
    )
    test_loader = DataLoader(
        test_data, batch_size=64,
        shuffle=True, drop_last=True
    )
    # Train 100 epochs adding the new data in
    train_net(
        model, train_loader, test_data,
        loss_fn=torch.nn.BCEWithLogitsLoss(),
        epochs=100, learning_rate=0.01
    )
    month_acc = pred_accuracy(model(test_data.x).squeeze(), test_data.y)
    accuracies.append(month_acc)

In [ ]:
plt.plot(accuracies)